In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
sys.path.append("..")
sys.path.append(".")
import torch
import numpy as np
import copy
import plotly
import plotly.graph_objs as go
import pickle
from datetime import datetime
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.samples_generator as samples_generator
import robust_value_approx.samples_buffer as samples_buffer
import robust_value_approx.value_approximation as value_approximation
import robust_value_approx.training_log as training_log
import robust_value_approx.controllers as controllers

import pendulum_utils
import double_pendulum_utils
import acrobot_utils
import manipulator_utils

In [10]:
dtype = torch.float64

sys_name = 'manipulator'
opt = dict(
    dtype = dtype,
    
    sys_name = sys_name,
    validation_file = '../data/validation_' + sys_name,
    init_file = '../data/init_' + sys_name,
    
    offline_horizon = 20,
    offline_dt = .1,
    
    sys_utils = manipulator_utils,
)

In [ ]:
vf, sys = opt['sys_utils'].get_value_function(opt['offline_horizon'], opt['offline_dt'], dtype=opt['dtype'])
V = vf.get_value_function()
v, res = V(opt['sim_x0'])
sys.plot_result(res).show()

In [2]:
import numpy as np
import os
import time

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [3]:
# Create a new visualizer
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
vis.jupyter_cell()

In [5]:
finger_base = vis["finger_base"]
finger_joint = finger_base["finger_joint"]
finger = finger_joint["finger"]
eef = finger["eef"]
ellipsoid = vis["ellipsoid"]

finger_base.set_object(g.Box([.05, .05, .05]))
finger.set_object(g.Box([.5, 0.025, 0.025]))
eef.set_object(g.Sphere(0.025), g.MeshLambertMaterial(color=0xff22dd, reflectivity=0.8))
ellipsoid.set_object(g.Ellipsoid([.25, 0.05, .15]), g.MeshLambertMaterial(color=0x5a23cd, reflectivity=0.8))

finger_base.set_transform(tf.translation_matrix([0, 0, .5]))
finger.set_transform(tf.translation_matrix([-.25, 0, 0]))
finger_joint.set_transform(tf.rotation_matrix(-np.pi/2, [1, 0, 0]))
eef.set_transform(tf.translation_matrix([-.25, 0, 0]))



In [13]:
for x in np.linspace(-np.pi, np.pi, 200):
    finger_base.set_transform(tf.translation_matrix([.5 * x / np.pi, 0, .5]))
    finger_joint.set_transform(tf.rotation_matrix(-x / 4 - np.pi / 2, [0, 1, 0]))
    ellipsoid.set_transform(tf.rotation_matrix(-x / 4 - np.pi / 2, [0, 1, 0]))
    time.sleep(0.005)